In [1]:

from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
!pip install evaluate
import evaluate
from sklearn.model_selection import train_test_split



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


In [2]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [3]:
model_small= 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_small)
model = T5ForConditionalGeneration.from_pretrained(model_small)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
!pip install datasets

In [5]:
#I am importing BioASQ dataset to fine-tune the t5 model on question answering task
from datasets import load_dataset
dataset = load_dataset("kroshan/BioASQ")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
answers = []
contexts = []

for text in dataset["train"]["text"]:
    answer, context = text.split("<answer>")[1].split("<context>")
    answers.append(answer.strip())
    contexts.append(context.strip())

dataset["train"] = dataset["train"].add_column("answer", answers)
dataset["train"] = dataset["train"].add_column("context", contexts)

In [7]:
import pandas as pd

In [8]:
data = pd.DataFrame(dataset["train"])
data = data.drop(columns=["text"])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3266 entries, 0 to 3265
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  3266 non-null   object
 1   answer    3266 non-null   object
 2   context   3266 non-null   object
dtypes: object(3)
memory usage: 76.7+ KB


In [10]:
data

,question,answer,context
0,What is the inheritance pattern of Li–Fraumeni...,autosomal dominant,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...
1,What is the inheritance pattern of Li–Fraumeni...,autosomal dominant,Genetic modeling of Li-Fraumeni syndrome in ze...
2,Which type of lung cancer is afatinib used for?,EGFR-mutant NSCLC,Clinical perspective of afatinib in non-small ...
3,Which hormone abnormalities are characteristic...,thyroid,"DOCA sensitive pendrin expression in kidney, h..."
4,Which hormone abnormalities are characteristic...,thyroid,Clinical and molecular characteristics of Pend...
...,...,...,...
3261,Which is the receptor for substrates of Chaper...,lamp2a,Regulation of lamp2a levels in the lysosomal m...
3262,Which is the receptor for substrates of Chaper...,Lamp2a,Unique properties of lamp2a compared to other ...
3263,Which is the receptor for substrates of Chaper...,lamp2a,Unique properties of lamp2a compared to other ...
3264,Which is the receptor for substrates of Chaper...,LAMP2A,Dietary lipids and aging compromise chaperone-...


In [11]:
from torch.utils.data import Dataset, DataLoader, RandomSampler


In [12]:
import torch

def tokenize_data(data, tokenizer):
    tokenized_data = []
    for ind in range(len(data)):
        question = data["question"][ind]
        context = data["context"][ind]
        answer = data['answer'][ind]

        questionTk = tokenizer(question, context, max_length=256, padding="max_length",
                                       truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answerTk = tokenizer(answer, max_length=32, padding="max_length",
                                     truncation=True, pad_to_max_length=True, add_special_tokens=True)

        lb = torch.tensor(answerTk["input_ids"], dtype=torch.long)
        lb[lb == 0] = -100

        tokenized_data.append({
            "input_ids": torch.tensor(questionTk["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(questionTk["attention_mask"], dtype=torch.long),
            "labels": lb,
            "decoder_attention_mask": torch.tensor(answerTk["attention_mask"], dtype=torch.long)
        })

    return tokenized_data


In [13]:
trainData, testData = train_test_split(data, test_size=0.2, random_state=42)

train_rand_sampler = RandomSampler(trainData.index)
test_rand_sampler = RandomSampler(testData.index)

tokenized_data = tokenize_data(data, tokenizer)

train_loader_output = DataLoader(tokenized_data, batch_size=4, sampler=train_rand_sampler)
test_loader_output = DataLoader(tokenized_data, batch_size=4, sampler=test_rand_sampler)

In [14]:
from tqdm import tqdm
from torch.optim import Adam

In [15]:
import torch.optim as optim

epochs = 2
optimizer = optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_batch_count = 0

    for batch in tqdm(train_loader_output, desc="train batches"):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )

        optimizer.zero_grad()
        outputs.loss.backward()
        optimizer.step()

    model.eval()
    for batch in tqdm(test_loader_output, desc="val batches"):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        outputs = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        decoder_attention_mask=decoder_attention_mask
                        )

        optimizer.zero_grad()
        outputs.loss.backward()
        optimizer.step()



val batches: 100%|██████████| 164/164 [11:12<00:00,  4.10s/it]


In [16]:
def predict_answer_score(context, question, ref_answer=None):
    inputs = tokenizer(question, context, max_length=256, padding="max_length", truncation=True, return_tensors="pt", add_special_tokens=True)

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer], references=[ref_answer])

        return {
            "reference": ref_answer,
            "prediction": predicted_answer,
            "bleu score": score
        }
    else:
        return predicted_answer


In [17]:
import numpy as np

In [18]:
bleu_scores = []
i=0
data_500 = data.head(500)
for start_row in range(0, len(data_500)):
    row = data_500.iloc[start_row]

    context = row["context"]
    question = row["question"]
    answer = row["answer"]
    score = predict_answer_score(context, question, answer)

    bleu_scores.append(score["bleu score"]['google_bleu'])
    if i<30 and i>10:
      print(score)
    i+=1



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'bleu score': {'google_bleu': 1.0}}
{'reference': 'thyroid', 'prediction': 'thyroid', 'ble

In [19]:
average_bleu_score = np.mean(bleu_scores)
print(f"Average BLEU Score of 500 rows: {average_bleu_score}")

Average BLEU Score of 500 rows: 0.698892188016326


In [20]:
context = data.iloc[21]["context"]
question = data.iloc[21]["question"]
answer = data.iloc[21]["answer"]

predict_answer_score(context, question, answer)

{'reference': 'thyroid',
 'prediction': 'thyroid',
 'bleu score': {'google_bleu': 1.0}}

In [21]:
#Save model and tokenizer to your Hugging Face account
model.save_pretrained("starman76/t5_bioasq_QA")
tokenizer.save_pretrained("starman76/t5_bioasq_QA")


('starman76/t5_bioasq_QA/tokenizer_config.json',
 'starman76/t5_bioasq_QA/special_tokens_map.json',
 'starman76/t5_bioasq_QA/spiece.model',
 'starman76/t5_bioasq_QA/added_tokens.json')

In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [25]:
model.push_to_hub("starman76/t5_500")
tokenizer.push_to_hub("starman76/t5_500")


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/starman76/t5_500/commit/3e7abad96fa3001d05c3e799b06202469873c3cf', commit_message='Upload tokenizer', commit_description='', oid='3e7abad96fa3001d05c3e799b06202469873c3cf', pr_url=None, pr_revision=None, pr_num=None)